In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
t = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
t.head(10)

In [ ]:
t.count()

On remarque que toutes les données sont complètes. Ceci n'est pas contre pas vraiment le cas, parce que à chaque fois qu'une caractéristique est inconnue ils ont mis 0 comme valeur (une BloodPressure = 0 ne fait pas trop de sens si le patient est encore vivant).

Je vais donc essayer de remplacer les 0 dans les colonnes voulues (pas remplacer les 0 dans Outcome bien sûr) par la moyenne des valeurs comme on l'a fait dans l'exemple titanic.

L'idée est de remplacer tout les 0 par la moyenne de la colonne:

In [ ]:
t.BloodPressure = t.BloodPressure.replace([0],t.BloodPressure.mean())

In [ ]:
t.head(20)

On vient donc de remplacer tout les 0 de la colonne BloodPressure par la moyenne de la colonne. On refait cela pour les colonnes restantes:

In [ ]:
t.SkinThickness = t.SkinThickness.replace([0],t.SkinThickness.mean())
t.Insulin = t.Insulin.replace([0],t.Insulin.mean())
t.BMI = t.BMI.replace([0],t.BMI.mean())

In [ ]:
t.head(20)

Maintenant que le tableau est rempli, on peut continuer avec la partie machine learning:

In [ ]:
t.Outcome.value_counts()

In [ ]:
nb_malade = (t.Outcome == 1)

In [ ]:
t[nb_malade].head(10)

In [ ]:
prob = (t[nb_malade].Outcome.count())/(t.Outcome.count())
print(prob)

34,9% des patients dans cette liste sont diabétique.

Je ne constate pas un déséquilibre des distributions puisque les donnés dépendent vraiment de chaque personne. De plus, une transformation en binaire me parait pas réalisable. Du coup, je continue avec la prédiction de patients positifs à l'aide du machine learning:  

In [ ]:
data_train = t.sample(frac=0.8, random_state=1)
data_test = t.drop(data_train.index)

In [ ]:
X_train = data_train.drop(['Outcome'], axis=1)
y_train = data_train['Outcome']
X_test = data_test.drop(['Outcome'], axis=1)
y_test = data_test['Outcome']

On aplique le principe de la regression logistique:

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)

In [ ]:
cm = confusion_matrix(y_test, y_lr)
print(cm)

On obtient un résultat pas tellement satisfaisant (76,6%). La méthode de la regression logistique n'est donc peut-être pas la bonne méthode à choisir. On essayera d'appliquer la méthode des arbres de décision:

In [ ]:
sns.pairplot(t, hue="Outcome")

On constate que les courbes sont tous l'un sur l'autre. Une séparation par arbres de décision va donc être difficile. On constate également un déséquilibre des distributions des courbes 'Insulin' (l'hypothèse pris au début qu'il y en a pas de déséquiibre était donc faux). Je vais donc faire une transformation en log pour 'Insulin':  

In [ ]:
sns.kdeplot(t.Insulin, color='blue')

In [ ]:
t['log_Insulin'] = np.log(t.Insulin+1)

In [ ]:
t.head()

In [ ]:
sns.kdeplot(t.log_Insulin, color='blue')

In [ ]:
t = t.drop(['Insulin'], axis = 1)

In [ ]:
t.head()

On refait la regression logistique:

In [ ]:
data_train = t.sample(frac=0.8, random_state=1)
data_test = t.drop(data_train.index)

In [ ]:
X_train = data_train.drop(['Outcome'], axis=1)
y_train = data_train['Outcome']
X_test = data_test.drop(['Outcome'], axis=1)
y_test = data_test['Outcome']

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

In [ ]:
lr_score = accuracy_score(y_test, y_lr)
print(lr_score)

In [ ]:
cm = confusion_matrix(y_test, y_lr)
print(cm)

On constate que le résultat c'est empiré (maintenant 75,3%, avant 76,6%). Je ne sais pas comment améliorer ce résultat. En regardant les diférentes distributions on a déjà constaté que les caractéristiques sont tous l'un sur l'autre, je peux donc m'imaginer que l'algorithme a des difficultés à déterminer le Outcome. Je vais continuer avec la mesure de performance:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

In [ ]:
print(classification_report(y_test, y_lr))

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

In [ ]:
tprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
tprobas['y'] = np.array(y_test)

In [ ]:
tprobas

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-tprobas.proba_0[tprobas.y==0], bins=50)
sns.distplot(tprobas.proba_1[tprobas.y==1], bins=50)

On voit que la distribution est loin d'être idéale, comme déjà constaté avant. On continue avec la création de la courbe ROC:

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

On remarque que la courbe se trouve au milieu de la courbe idéale et de la courbe la moin bien.

Ajustement des hyperparamètres (Random Forests):

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

In [ ]:
X = t.drop(['Outcome'], axis=1)
y = t.Outcome

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 2)
plt.xlabel('n_estimators')
plt.ylabel('score');

La méthode GridSearchCV:

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

J'ai l'impression que cette méthode n'a pas pu grandemant améliorer le modèle, au contraire, les valeurs ont diminués.

Pour finaliser ce Notebook, voici le classement de l'importance des différentes caractéristiques:

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')

**Annexe XGBoost:**

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

A nouveau pas d'amélioration.